# PTT 網路爬蟲實作練習


* 能夠利用 Request + BeatifulSour 撰寫爬蟲，並存放到合適的資料結構


## 作業目標

根據範例 ，完成以下問題：

* ① 印出最新文章的「作者」「標題」「時間」
* ② 印出第一頁所有文章的「作者」「標題」「時間」


In [7]:
import requests
from bs4 import BeautifulSoup

url = 'https://www.ptt.cc/bbs/NBA/index.html'
r = requests.get(url)

soup = BeautifulSoup(r.text, "html5lib")

for d in soup.find_all(class_="title"):
    #print(d.text)
    print(d.text.replace('\t', '').replace('\n', ''))  #去掉換行(\n)與Tab(\t)
    try:
        r = BeautifulSoup(requests.get('https://www.ptt.cc'+d.find('a')['href']).text, "html5lib")
        print('作者: ' + r.find(class_='article-meta-value').text)
    except:
        continue

[情報] 狀元:我感覺我打得非常好,適應NBA並不難
作者: sezna (sezna)
[BOX ] Jazz 120:100 Blazers 數據
[討論] Buddy絕殺後帶領隊友狂奔
作者: humbler (獸人H)
[BOX ] Mavericks 102:106 Suns 數據
[情報] Kerr:開幕戰收到KT的訊息，需要這種聯繫
作者: ckny (辛苦的歲月)
[花邊] Jaylen與Tatum合照慶祝勝利：打板三分，
作者: ckny (辛苦的歲月)
Re: [BOX] Mavericks 102:106 Suns 數據
作者: yokoogawa (Hey)
[公告] 板規v6.8
[情報] 2020-21 自由球員市場異動 (表格)(12/10)
[情報] 2020-21 自由球員市場異動 (每日文字)  
[情報] PRE SEASON Schedule 賽程 20–21
作者: Acetoxy (阿斯)


### ① 印出最新文章的「作者」「標題」「時間」

In [12]:
# 因為原始的資料難以判斷「新/舊」，因此我們必須進入內頁抓取詳細的時間，最終存成 List of Dict 的形式再自行排序。

posts = []

for d in soup.find_all(class_="title"):
    try:
        post = {}
        post['標題'] = d.text.replace('\t', '').replace('\n', '')
        
        r = BeautifulSoup(requests.get('https://www.ptt.cc'+d.find('a')['href']).text, "html5lib")
        post['作者'] = r.find(class_='article-meta-value').text
        post['時間'] = r.find_all(class_='article-meta-value')[-1].text
        
        posts.append(post)
    except:
        continue
        
posts = sorted(posts, key= lambda x: x['時間']) #預設 reverse=False，順序排列，最後一筆為最新資料
posts_2 = sorted(posts, key= lambda x: x['時間'], reverse=False)
posts_3 = sorted(posts, key= lambda x: x['時間'], reverse=True) #倒序排列，第一筆為最新資料

# 補充：List of Dict 的排序方法
# https://stackoverflow.com/questions/72899/how-do-i-sort-a-list-of-dictionaries-by-a-value-of-the-dictionary
print(posts[-1])
print(posts_2[-1])
print(posts_3[-1])

{'標題': 'Re: [BOX] Mavericks 102:106 Suns 數據', '作者': 'yokoogawa (Hey)', '時間': 'Thu Dec 24 16:00:05 2020'}
{'標題': 'Re: [BOX] Mavericks 102:106 Suns 數據', '作者': 'yokoogawa (Hey)', '時間': 'Thu Dec 24 16:00:05 2020'}
{'標題': '[情報] PRE SEASON Schedule 賽程 20–21', '作者': 'Acetoxy (阿斯)', '時間': 'Sat Dec 12 01:59:46 2020'}


### ② 印出第一頁所有文章的「作者」「標題」「時間」

In [13]:
# 方法一：只從首頁
for d in soup.find_all(class_="r-ent"):
    print('標題： ', d.find(class_='title').text.replace('\t', '').replace('\n', ''))
    print('作者： ', d.find(class_='author').text.replace('\t', '').replace('\n', ''))
    print('時間： ', d.find(class_='date').text.replace('\t', '').replace('\n', ''))
    

標題：  [情報] 狀元:我感覺我打得非常好,適應NBA並不難
作者：  sezna
時間：  12/24
標題：  [BOX ] Jazz 120:100 Blazers 數據
作者：  Rambo
時間：  12/24
標題：  [討論] Buddy絕殺後帶領隊友狂奔
作者：  humbler
時間：  12/24
標題：  [BOX ] Mavericks 102:106 Suns 數據
作者：  Rambo
時間：  12/24
標題：  [情報] Kerr:開幕戰收到KT的訊息，需要這種聯繫
作者：  ckny
時間：  12/24
標題：  [花邊] Jaylen與Tatum合照慶祝勝利：打板三分，
作者：  ckny
時間：  12/24
標題：  Re: [BOX] Mavericks 102:106 Suns 數據
作者：  yokoogawa
時間：  12/24
標題：  [公告] 板規v6.8
作者：  Vedan
時間：   5/24
標題：  [情報] 2020-21 自由球員市場異動 (表格)(12/10)
作者：  laigei
時間：  11/19
標題：  [情報] 2020-21 自由球員市場異動 (每日文字)  
作者：  laigei
時間：  11/19
標題：  [情報] PRE SEASON Schedule 賽程 20–21
作者：  Acetoxy
時間：  12/12


In [14]:
# 方法二：進入內頁抓
for d in soup.find_all(class_="title"):
    try:
        print('標題： ', d.text.replace('\t', '').replace('\n', ''))
        
        r = BeautifulSoup(requests.get('https://www.ptt.cc'+d.find('a')['href']).text, "html5lib")
        print('作者： ' + r.find(class_='article-meta-value').text)
        print('時間： ' + r.find_all(class_='article-meta-value')[-1].text)
    except:
        continue

標題：  [情報] 狀元:我感覺我打得非常好,適應NBA並不難
作者： sezna (sezna)
時間： Thu Dec 24 13:14:44 2020
標題：  [BOX ] Jazz 120:100 Blazers 數據
標題：  [討論] Buddy絕殺後帶領隊友狂奔
作者： humbler (獸人H)
時間： Thu Dec 24 13:26:06 2020
標題：  [BOX ] Mavericks 102:106 Suns 數據
標題：  [情報] Kerr:開幕戰收到KT的訊息，需要這種聯繫
作者： ckny (辛苦的歲月)
時間： Thu Dec 24 14:37:41 2020
標題：  [花邊] Jaylen與Tatum合照慶祝勝利：打板三分，
作者： ckny (辛苦的歲月)
時間： Thu Dec 24 14:55:38 2020
標題：  Re: [BOX] Mavericks 102:106 Suns 數據
作者： yokoogawa (Hey)
時間： Thu Dec 24 16:00:05 2020
標題：  [公告] 板規v6.8
標題：  [情報] 2020-21 自由球員市場異動 (表格)(12/10)
標題：  [情報] 2020-21 自由球員市場異動 (每日文字)  
標題：  [情報] PRE SEASON Schedule 賽程 20–21
作者： Acetoxy (阿斯)
時間： Sat Dec 12 01:59:46 2020


### ③ 試著爬爬看其他版的文章

In [16]:
# 改成 Lifeismoney 版，抓出最新一筆的文章

import requests
from bs4 import BeautifulSoup

url = 'https://www.ptt.cc/bbs/Lifeismoney/index.html'
r = requests.get(url)

soup = BeautifulSoup(r.text, "html5lib")

posts = []

for d in soup.find_all(class_="title"):
    try:
        post = {}
        post['標題'] = d.text.replace('\t', '').replace('\n', '')
        
        r = BeautifulSoup(requests.get('https://www.ptt.cc'+d.find('a')['href']).text, "html5lib")
        post['作者'] = r.find(class_='article-meta-value').text
        post['時間'] = r.find_all(class_='article-meta-value')[-1].text
        
        posts.append(post)
    except:
        continue
        
posts = sorted(posts, key= lambda x: x['時間'])
posts[-1]

{'標題': '[公告] 未滿1元之集點資訊集中文',
 '作者': 'medama ( )',
 '時間': 'Tue Dec  1 23:53:51 2020'}

In [23]:
# 改成 Gossiping 版，發生錯誤，因為八卦版會跳轉到一個「十八歲的同意驗證頁面」導致錯誤。

import requests
from bs4 import BeautifulSoup

url = 'https://www.ptt.cc/bbs/Gossiping/index.html'
r = requests.get(url)

soup = BeautifulSoup(r.text, "html5lib")

posts = []

for d in soup.find_all(class_="title"):
    try:
        post = {}
        post['標題'] = d.text.replace('\t', '').replace('\n', '')
        
        r = BeautifulSoup(requests.get('https://www.ptt.cc'+d.find('a')['href']).text, "html5lib")
        post['作者'] = r.find(class_='article-meta-value').text
        post['時間'] = r.find_all(class_='article-meta-value')[-1].text
        
        posts.append(post)
    except:
        continue
        
posts = sorted(posts, key= lambda x: x['時間'])
posts[-1]

IndexError: list index out of range

In [24]:
# 改成 Gossiping 版，發生錯誤，因為八卦版會跳轉到一個「十八歲的同意驗證頁面」導致錯誤。
# 參考圖片下載時的解法，加上 cookies 繞過驗證（後面課程會再進行補充）

import requests
from bs4 import BeautifulSoup

url = 'https://www.ptt.cc/bbs/Gossiping/index.html'
r = requests.get(url, cookies={'over18': '1'})

soup = BeautifulSoup(r.text, "html5lib")

posts = []

for d in soup.find_all(class_="title"):
    try:
        post = {}
        post['標題'] = d.text.replace('\t', '').replace('\n', '')
        
        r = BeautifulSoup(requests.get('https://www.ptt.cc'+d.find('a')['href'], cookies={'over18': '1'}).text, "html5lib")
        post['作者'] = r.find(class_='article-meta-value').text
        post['時間'] = r.find_all(class_='article-meta-value')[-1].text
        
        posts.append(post)
    except:
        continue
        
posts = sorted(posts, key= lambda x: x['時間'])
posts[-1]

{'標題': '[公告] 線上聖誕夜活動閒聊&活動專區',
 '作者': 'ubcs (覺★青年超冒險蓋)',
 '時間': 'Tue Dec 22 20:12:50 2020'}

In [32]:
# 我們發現這個時間好像不是正確的，原因是我們前面存到的時間，其實不是正確的格式

import requests
from bs4 import BeautifulSoup

url = 'https://www.ptt.cc/bbs/Gossiping/index.html'
r = requests.get(url, cookies={'over18': '1'})

soup = BeautifulSoup(r.text, "html5lib")

posts = []

for d in soup.find_all(class_="title"):
    try:
        post = {}
        post['標題'] = d.text.replace('\t', '').replace('\n', '')
        
        r = BeautifulSoup(requests.get('https://www.ptt.cc'+d.find('a')['href'], cookies={'over18': '1'}).text, "html5lib")
        post['作者'] = r.find(class_='article-meta-value').text
        post['時間'] = r.find_all(class_='article-meta-value')[-1].text
        #post['時間'] = datetime.strptime(post['時間'], "%a %b %d %H:%M:%S %Y")
        # 時間轉換：https://stackoverflow.com/questions/10256093/how-to-convert-ctime-to-datetime-in-python
        
        posts.append(post)
    except:
        continue
        
posts = sorted(posts, key= lambda x: x['時間'])
posts

[{'標題': '[公告] 八卦板板規(2020.11.21)',
  '作者': 'arsonlolita (蘿莉都布羅利)',
  '時間': 'Sat Nov 21 23:11:42 2020'},
 {'標題': '[協尋] 11/23晚間 東勢往卓蘭行車記錄器',
  '作者': 'ding88510 (丁丁)',
  '時間': 'Sat Nov 28 01:54:24 2020'},
 {'標題': '[問卦] 健康滷肉飯去哪裡吃？',
  '作者': 'TCMRYUTARO (TCMRYUTARO)',
  '時間': 'Thu Dec 24 20:52:25 2020'},
 {'標題': 'Re: [問卦] 禿頭是不是可以信伊斯蘭教來掩飾',
  '作者': 'drogoman (玉花驄)',
  '時間': 'Thu Dec 24 20:53:00 2020'},
 {'標題': '[問卦] 為啥臺灣沒有老罈酸菜牛肉麵',
  '作者': 'ltony01 (ltony01)',
  '時間': 'Thu Dec 24 20:53:55 2020'},
 {'標題': '[問卦] 陳瑞振或陳江和打的過馬保國嗎？',
  '作者': 'p69005103 (ANDY)',
  '時間': 'Thu Dec 24 20:54:17 2020'},
 {'標題': '[問卦] 猛甲和狂甲哪個好',
  '作者': 'InDer5566 (在這5566)',
  '時間': 'Thu Dec 24 20:54:44 2020'},
 {'標題': '[問卦] 這題這樣選可以嗎 ?',
  '作者': 'triumphant10 ()',
  '時間': 'Thu Dec 24 20:55:24 2020'},
 {'標題': '[問卦] 今際之國最正女角色要選誰?',
  '作者': 'coga4712 (海森堡)',
  '時間': 'Thu Dec 24 20:55:54 2020'},
 {'標題': '[問卦] 客戶疑似想挖角我  如何處置？',
  '作者': 'cc02040326 (茶包)',
  '時間': 'Thu Dec 24 20:56:50 2020'},
 {'標題': 'Re: [問卦] 朋友女友突然說今晚可以內射',
  

In [34]:
posts[-1]

{'標題': '[公告] 線上聖誕夜活動閒聊&活動專區',
 '作者': 'ubcs (覺★青年超冒險蓋)',
 '時間': 'Tue Dec 22 20:12:50 2020'}